In [3]:
import os
import requests
import pandas as pd
from datetime import datetime
import urllib3

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# Crear estructura de carpetas local (reproducible)
BASE_DIR = "/content/emergencias123_u4"
RAW_DIR = os.path.join(BASE_DIR, "raw")

os.makedirs(RAW_DIR, exist_ok=True)

print("Directorio preparado:", BASE_DIR)


Directorio preparado: /content/emergencias123_u4


In [4]:
PACKAGE_URL = "https://datosabiertos.bogota.gov.co/api/3/action/package_show"
DATASET_NAME = "llamadas-de-urgencias-y-emergencias-que-ingresan-a-traves-de-la-linea-123"

response = requests.get(PACKAGE_URL, params={"id": DATASET_NAME}, verify=False).json()

resources = response["result"]["resources"]
print("Total recursos encontrados:", len(resources))


Total recursos encontrados: 79


In [5]:
MESES_MAP = {
    "ENERO": 1, "FEBRERO": 2, "MARZO": 3, "ABRIL": 4, "MAYO": 5, "JUNIO": 6,
    "JULIO": 7, "AGOSTO": 8, "SEPTIEMBRE": 9, "OCTUBRE": 10, "NOVIEMBRE": 11, "DICIEMBRE": 12
}

def parse_fecha(nombre):
    nombre = nombre.upper().replace(".", " ")
    partes = nombre.split()
    mes = None
    anio = None
    for p in partes:
        if p in MESES_MAP:
            mes = MESES_MAP[p]
        if p.isdigit() and len(p) == 4:
            anio = int(p)
    if mes and anio:
        return datetime(anio, mes, 1)
    return datetime(1900, 1, 1)

recursos_mensuales = [
    (r["name"], r["id"])
    for r in resources
    if "SALUD" in r["name"].upper()
    and any(m in r["name"].upper() for m in MESES_MAP)
]

# Ordenar por fecha descendente
recursos_ordenados = sorted(recursos_mensuales, key=lambda x: parse_fecha(x[0]), reverse=True)

# Tomar los 12 meses más recientes
recursos_12 = recursos_ordenados[:12]

print("Últimos 12 recursos detectados:")
for n, rid in recursos_12:
    print("-", n, "=>", rid)


Últimos 12 recursos detectados:
- LLAMADAS DE URGENCIAS Y EMERGENCIAS LINEA 123 - SALUD OCTUBRE 2025. => 9b5c7c88-7176-47fa-b82f-f03c5af26594
- LLAMADAS DE URGENCIAS Y EMERGENCIAS LINEA 123 - SALUD SEPTIEMBRE 2025. => 8271e353-b9ba-497a-acbc-3c92d3edd52e
- LLAMADAS DE URGENCIAS Y EMERGENCIAS LINEA 123 - SALUD AGOSTO 2025. => d164dfcc-add8-4f3a-8f55-d10e0ab6bed3
- LLAMADAS DE URGENCIAS Y EMERGENCIAS LINEA 123 - SALUD JULIO 2025. => fa221f24-593a-4d3c-b697-e2f42ccbb314
- LLAMADAS DE URGENCIAS Y EMERGENCIAS LINEA 123 - SALUD JUNIO 2025 => 15b8faf8-60e3-451a-98bc-090b69411698
- LLAMADAS DE URGENCIAS Y EMERGENCIAS LINEA 123 - SALUD MAYO 2025 => a71dc0c9-6d38-4bcf-964d-7eea5b24aeb1
- LLAMADAS DE URGENCIAS Y EMERGENCIAS LINEA 123 - SALUD ABRIL 2025 => 9867ba1a-bd16-41b0-97f1-52156d4fdf32
- LLAMADAS DE URGENCIAS Y EMERGENCIAS LINEA 123 - SALUD MARZO 2025 => f9e22870-ae80-4a3b-98a7-05e3de90435c
- LLAMADAS DE URGENCIAS Y EMERGENCIAS LINEA 123 - SALUD FEBRERO 2025 => 75bfeee8-abfe-4505-83f4-cdc1c

In [29]:
DATASTORE_URL = "https://datosabiertos.bogota.gov.co/api/3/action/datastore_search"
dfs = []

for nombre, rid in recursos_12:
    offset = 0
    limit = 50000
    registros = []

    while True:
        params = {"resource_id": rid, "limit": limit, "offset": offset}
        resp = requests.get(DATASTORE_URL, params=params, verify=False).json()

        batch = resp["result"]["records"]
        if not batch:
            break

        registros.extend(batch)
        offset += limit

    print("   Registros obtenidos:", len(registros))

    df_mes = pd.DataFrame(registros)
    df_mes["mes_origen"] = nombre
    dfs.append(df_mes)

df_raw = pd.concat(dfs, ignore_index=True)
print("Total registros consolidados:", len(df_raw))


   Registros obtenidos: 11008
   Registros obtenidos: 11433
   Registros obtenidos: 10603
   Registros obtenidos: 9868
   Registros obtenidos: 9766
   Registros obtenidos: 10703
   Registros obtenidos: 10706
   Registros obtenidos: 11626
   Registros obtenidos: 10497
   Registros obtenidos: 10246
   Registros obtenidos: 12035
   Registros obtenidos: 11159
Total registros consolidados: 129650


In [7]:
df = df_raw.copy()

rename_map = {
    "NUMERO_INCIDENTE": "numero_incidente",
    "FECHA_INICIO_DESPLAZAMIENTO_MOVIL": "fecha_inicio_desplazamiento_movil",
    "CODIGO_LOCALIDAD": "codigo_localidad",
    "LOCALIDAD": "localidad",
    "EDAD": "edad",
    "UNIDAD": "unidad",
    "GENERO": "genero",
    "TIPO_INCIDENTE": "tipo_incidente",
    "PRIORIDAD_FINAL": "prioridad_final",
    "RECEPCION": "recepcion",
}

df = df.rename(columns={k:v for k,v in rename_map.items() if k in df.columns})

# Convertir fechas
if "fecha_inicio_desplazamiento_movil" in df.columns:
    df["fecha_inicio_desplazamiento_movil"] = pd.to_datetime(df["fecha_inicio_desplazamiento_movil"], errors="coerce")

# Columnas anio/mes
df["anio"] = df["fecha_inicio_desplazamiento_movil"].dt.year
df["mes"] = df["fecha_inicio_desplazamiento_movil"].dt.month

df.head()


,_id,numero_incidente,fecha_inicio_desplazamiento_movil,codigo_localidad,localidad,edad,unidad,genero,tipo_incidente,prioridad_final,recepcion,mes_origen,FECHA_INICIO_DESPLAZAMIENTO_MOVIl,PRIORIDAD FINAL,anio,mes
0,1,CRU-00522381-25,2025-10-01 02:50:09,14,LOS MµRTIRES,23,A§os,FEMENINO,ACOMPA•AMIENTO A EVENTO,Baja,None,LLAMADAS DE URGENCIAS Y EMERGENCIAS LINEA 123 ...,NaN,NaN,2025.0,10.0
1,2,CRU-00522381-25,2025-10-01 00:04:24,14,LOS MµRTIRES,N/A,N/A,N/A,ACOMPA•AMIENTO A EVENTO,Baja,None,LLAMADAS DE URGENCIAS Y EMERGENCIAS LINEA 123 ...,NaN,NaN,2025.0,10.0
2,3,CRU-00522382-25,2025-10-01 00:14:19,9,FONTIB‡N,23,A§os,FEMENINO,PATGIN - PATOLOG÷A GINECOBSTêTRICA,Critica,2025-10-01 02:38:04,LLAMADAS DE URGENCIAS Y EMERGENCIAS LINEA 123 ...,NaN,NaN,2025.0,10.0
3,4,CRU-00522388-25,2025-10-01 00:46:30,1,USAQUêN,86,A§os,FEMENINO,EVERES - EVENTO RESPIRATORIO,Alta,2025-10-01 04:53:42,LLAMADAS DE URGENCIAS Y EMERGENCIAS LINEA 123 ...,NaN,NaN,2025.0,10.0
4,5,CRU-00522391-25,2025-10-01 00:41:05,10,ENGATIVµ,N/A,N/A,N/A,ACOMPA•AMIENTO A EVENTO,Baja,None,LLAMADAS DE URGENCIAS Y EMERGENCIAS LINEA 123 ...,NaN,NaN,2025.0,10.0


In [30]:
csv_path = os.path.join(RAW_DIR, "emergencias_123_12meses_normalizado.csv")
df.to_csv(csv_path, index=False, encoding="utf-8")

print("Dataset normalizado guardado en:")
print(csv_path)


Dataset normalizado guardado en:
/content/emergencias123_u4/raw/emergencias_123_12meses_normalizado.csv


In [27]:
df = pd.read_csv("/content/emergencias123_u4/raw/emergencias_123_12meses_normalizado.csv")

df.head()

/tmp/ipython-input-2869154717.py:1: DtypeWarning:

Columns (5,12,13) have mixed types. Specify dtype option on import or set low_memory=False.



,_id,numero_incidente,fecha_inicio_desplazamiento_movil,codigo_localidad,localidad,edad,unidad,genero,tipo_incidente,prioridad_final,recepcion,mes_origen,FECHA_INICIO_DESPLAZAMIENTO_MOVIl,PRIORIDAD FINAL,anio,mes
0,1,CRU-00522381-25,2025-10-01 02:50:09,14,LOS MµRTIRES,23.0,A§os,FEMENINO,ACOMPA•AMIENTO A EVENTO,Baja,NaN,LLAMADAS DE URGENCIAS Y EMERGENCIAS LINEA 123 ...,NaN,NaN,2025.0,10.0
1,2,CRU-00522381-25,2025-10-01 00:04:24,14,LOS MµRTIRES,NaN,NaN,NaN,ACOMPA•AMIENTO A EVENTO,Baja,NaN,LLAMADAS DE URGENCIAS Y EMERGENCIAS LINEA 123 ...,NaN,NaN,2025.0,10.0
2,3,CRU-00522382-25,2025-10-01 00:14:19,9,FONTIB‡N,23.0,A§os,FEMENINO,PATGIN - PATOLOG÷A GINECOBSTêTRICA,Critica,2025-10-01 02:38:04,LLAMADAS DE URGENCIAS Y EMERGENCIAS LINEA 123 ...,NaN,NaN,2025.0,10.0
3,4,CRU-00522388-25,2025-10-01 00:46:30,1,USAQUêN,86.0,A§os,FEMENINO,EVERES - EVENTO RESPIRATORIO,Alta,2025-10-01 04:53:42,LLAMADAS DE URGENCIAS Y EMERGENCIAS LINEA 123 ...,NaN,NaN,2025.0,10.0
4,5,CRU-00522391-25,2025-10-01 00:41:05,10,ENGATIVµ,NaN,NaN,NaN,ACOMPA•AMIENTO A EVENTO,Baja,NaN,LLAMADAS DE URGENCIAS Y EMERGENCIAS LINEA 123 ...,NaN,NaN,2025.0,10.0


In [10]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go


df = pd.read_csv("/content/emergencias123_u4/raw/emergencias_123_12meses_normalizado.csv")

# Convertir fecha a datetime si es necesario
df["fecha_inicio_desplazamiento_movil"] = pd.to_datetime(
    df["fecha_inicio_desplazamiento_movil"], errors="coerce"
)

df["hora"] = df["fecha_inicio_desplazamiento_movil"].dt.hour
df["dia_semana"] = df["fecha_inicio_desplazamiento_movil"].dt.day_name()
df["mes_nombre"] = df["fecha_inicio_desplazamiento_movil"].dt.month_name()

df.head()


/tmp/ipython-input-1142393076.py:5: DtypeWarning: Columns (5,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/content/emergencias123_u4/raw/emergencias_123_12meses_normalizado.csv")


,_id,numero_incidente,fecha_inicio_desplazamiento_movil,codigo_localidad,localidad,edad,unidad,genero,tipo_incidente,prioridad_final,recepcion,mes_origen,FECHA_INICIO_DESPLAZAMIENTO_MOVIl,PRIORIDAD FINAL,anio,mes,hora,dia_semana,mes_nombre
0,1,CRU-00522381-25,2025-10-01 02:50:09,14,LOS MµRTIRES,23.0,A§os,FEMENINO,ACOMPA•AMIENTO A EVENTO,Baja,NaN,LLAMADAS DE URGENCIAS Y EMERGENCIAS LINEA 123 ...,NaN,NaN,2025.0,10.0,2.0,Wednesday,October
1,2,CRU-00522381-25,2025-10-01 00:04:24,14,LOS MµRTIRES,NaN,NaN,NaN,ACOMPA•AMIENTO A EVENTO,Baja,NaN,LLAMADAS DE URGENCIAS Y EMERGENCIAS LINEA 123 ...,NaN,NaN,2025.0,10.0,0.0,Wednesday,October
2,3,CRU-00522382-25,2025-10-01 00:14:19,9,FONTIB‡N,23.0,A§os,FEMENINO,PATGIN - PATOLOG÷A GINECOBSTêTRICA,Critica,2025-10-01 02:38:04,LLAMADAS DE URGENCIAS Y EMERGENCIAS LINEA 123 ...,NaN,NaN,2025.0,10.0,0.0,Wednesday,October
3,4,CRU-00522388-25,2025-10-01 00:46:30,1,USAQUêN,86.0,A§os,FEMENINO,EVERES - EVENTO RESPIRATORIO,Alta,2025-10-01 04:53:42,LLAMADAS DE URGENCIAS Y EMERGENCIAS LINEA 123 ...,NaN,NaN,2025.0,10.0,0.0,Wednesday,October
4,5,CRU-00522391-25,2025-10-01 00:41:05,10,ENGATIVµ,NaN,NaN,NaN,ACOMPA•AMIENTO A EVENTO,Baja,NaN,LLAMADAS DE URGENCIAS Y EMERGENCIAS LINEA 123 ...,NaN,NaN,2025.0,10.0,0.0,Wednesday,October


In [25]:
df_filtrado = df.copy()

# Aplicar filtros...
df_filtrado = df_filtrado[df_filtrado["localidad"].isin(["KENNEDY", "SUBA"])]

inc_mes = df_filtrado.groupby("mes_nombre").size().reset_index(name="total")
inc_mes = inc_mes.sort_values("total")

fig1 = px.line(
    inc_mes,
    x="mes_nombre",
    y="total",
    markers=True,
    title="Cantidad de incidentes por mes (Filtrado)",
)
fig1.show()


Este gráfico permite identificar estacionalidad y meses con mayor carga operativa para la línea 123.

In [17]:
# Filtrar incidentes de salud mental
df_salud_mental = df[df["tipo_incidente"].str.contains("MENTAL", case=False, na=False)]

# Agrupar por mes
mental_mes = df_salud_mental.groupby("mes_nombre").size().reset_index(name="total")
mental_mes = mental_mes.sort_values("total")

# Gráfica
fig = px.line(
    mental_mes,
    x="mes_nombre",
    y="total",
    title="Evolución mensual de incidentes de salud mental",
    markers=True
)
fig.show()


La serie muestra la evolución de incidentes relacionados con salud mental y posibles tendencias estacionales.


In [12]:
inc_tipo = df.groupby("tipo_incidente").size().reset_index(name="total")
inc_tipo = inc_tipo.sort_values("total", ascending=False).head(10)

fig2 = px.bar(
    inc_tipo,
    x="tipo_incidente",
    y="total",
    title="Top 10 tipos de incidentes reportados",
)
fig2.show()


Las barras permiten comparar de forma clara cuáles tipos de incidentes son más frecuentes.

In [13]:
loc = df.groupby("localidad").size().reset_index(name="total")
loc = loc.sort_values("total", ascending=False)

fig3 = px.bar(
    loc,
    x="localidad",
    y="total",
    title="Incidentes por localidad"
)
fig3.show()


Este gráfico evidencia la concentración geográfica de la demanda y permite identificar zonas críticas.

In [14]:
fig4 = px.pie(
    df,
    names="genero",
    title="Distribución de género en incidentes"
)
fig4.show()


El gráfico circular muestra la participación relativa de cada género en los registros de emergencias.


In [15]:
heat = df.groupby(["dia_semana", "hora"]).size().reset_index(name="total")

fig5 = px.density_heatmap(
    heat,
    x="hora",
    y="dia_semana",
    z="total",
    title="Heatmap de incidentes por hora y día",
    color_continuous_scale="Inferno"
)
fig5.show()


El heatmap revela horas pico y patrones semanales que impactan la gestión de recursos operativos.

In [16]:
prio = df.groupby(["tipo_incidente", "prioridad_final"]).size().reset_index(name="total")

fig6 = px.bar(
    prio,
    x="tipo_incidente",
    y="total",
    color="prioridad_final",
    title="Niveles de prioridad por tipo de incidente",
)
fig6.show()


Visualización clave para entender cómo se distribuyen las prioridades según el tipo de incidente.


In [28]:
from google.colab import files
files.download('/content/emergencias123_u4/raw/emergencias_123_12meses_normalizado.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>